In [3]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report, r2_score

from amb_sdk.sdk import DarwinSdk

In [4]:
#reading in social explorer data
dataset_name = 'SocialExplorer.csv'
socialExplorer = pd.read_csv(dataset_name)
socialExplorer.head()

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,SE_T128_001,SE_T128_002,SE_T128_009,SE_T128_010,SE_T128_003,SE_T128_004,SE_T128_005,SE_T128_006,SE_T128_007,SE_T128_008
0,10010201001,15000US010010201001,"Block Group 1, Census Tract 201, Autauga Count...","Block Group 1, Census Tract 201, Autauga Count...",al,150,0,ACSSF,5837,NaN,...,177,162,157,5,0,9,0,0,0,6
1,10010201002,15000US010010201002,"Block Group 2, Census Tract 201, Autauga Count...","Block Group 2, Census Tract 201, Autauga Count...",al,150,0,ACSSF,5838,NaN,...,652,598,583,15,22,5,0,0,27,0
2,10010202001,15000US010010202001,"Block Group 1, Census Tract 202, Autauga Count...","Block Group 1, Census Tract 202, Autauga Count...",al,150,0,ACSSF,5839,NaN,...,475,475,406,69,0,0,0,0,0,0
3,10010202002,15000US010010202002,"Block Group 2, Census Tract 202, Autauga Count...","Block Group 2, Census Tract 202, Autauga Count...",al,150,0,ACSSF,5840,NaN,...,451,451,439,12,0,0,0,0,0,0
4,10010203001,15000US010010203001,"Block Group 1, Census Tract 203, Autauga Count...","Block Group 1, Census Tract 203, Autauga Count...",al,150,0,ACSSF,5841,NaN,...,1098,1052,1002,50,3,0,0,0,0,43


In [5]:
#includes only relevant columns (modeShareSplit and Geo_FIPS code) from Social Explorer
modeShareAll = socialExplorer.iloc[:,[0, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]]
modeShareAll.head()

#creates new dataframe with % SOV
modeShareSOV = modeShareAll.iloc[:,[0,1,2]]
modeShareSOV.head()
percSOV = modeShareAll['SE_T128_009']/modeShareAll['SE_T128_001']
modeShareSOV['percSOV'] = percSOV

#new dataframe with Geo_FIPS and SOV only (ID and class)
print(list(modeShareSOV))
modeShareSOVPerc = modeShareSOV.iloc[:,[0,3]]
print(modeShareSOVPerc.head())




['Geo_FIPS', 'SE_T128_001', 'SE_T128_002', 'percSOV']
      Geo_FIPS   percSOV
0  10010201001  0.887006
1  10010201002  0.894172
2  10010202001  0.854737
3  10010202002  0.973392
4  10010203001  0.912568


C:\Users\Susan\Miniconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
#contains features
smartLocation = pd.read_csv("SmartLocationDB.csv")
smartLocation.head()
list(smartLocation)

['OBJECTID',
 'GEOID10',
 'TRFIPS',
 'CFIPS',
 'SFIPS',
 'CSA',
 'CSA_Name',
 'CBSA',
 'CBSA_Name',
 'CBSA_EMP',
 'CBSA_POP',
 'CBSA_WRK',
 'COUNTHU10',
 'TOTPOP10',
 'HH',
 'P_WRKAGE',
 'AUTOOWN0',
 'PCT_AO0',
 'AUTOOWN1',
 'PCT_AO1',
 'AUTOOWN2P',
 'PCT_AO2P',
 'WORKERS',
 'R_LOWWAGEWK',
 'R_MEDWAGEWK',
 'R_HIWAGEWK',
 'R_PCTLOWWAGE',
 'EMPTOT',
 'E5_RET10',
 'E5_OFF10',
 'E5_IND10',
 'E5_SVC10',
 'E5_ENT10',
 'E8_RET10',
 'E8_OFF10',
 'E8_IND10',
 'E8_SVC10',
 'E8_ENT10',
 'E8_ED10',
 'E8_HLTH10',
 'E8_PUB10',
 'E_FEDT10',
 'E_FEDRET10',
 'E_FEDOFF10',
 'E_FEDIND10',
 'E_FEDSVC10',
 'E_FEDENT10',
 'E_LOWWAGEW',
 'E_MEDWAGEW',
 'E_HIWAGEWK',
 'E_PCTLOWWA',
 'AC_TOT',
 'AC_WATER',
 'AC_LAND',
 'AC_UNPR',
 'D1A',
 'D1B',
 'D1C',
 'D1C5_Ret10',
 'D1C5_Off10',
 'D1C5_Ind10',
 'D1C5_Svc10',
 'D1C5_Ent10',
 'D1C8_Ret10',
 'D1C8_Off10',
 'D1C8_Ind10',
 'D1C8_Svc10',
 'D1C8_Ent10',
 'D1C8_Ed10',
 'D1C8_Hlth10',
 'D1C8_Pub10',
 'D1D',
 'D1_flag',
 'D2A_JPHH',
 'D2B_E5MIX',
 'D2B_E5MIXA',
 'D2

In [7]:
#contains different sets of features to keep
setOne = ['CSA_Name','CBSA_Name', 'P_WKRAGE','PCT_AO0','PCT_AO1','PCT_A02','R_PCTLOWWAGE',]
new = smartLocation[["OBJECTID","GEOID10"]].copy()

In [8]:
smartLocationOnlyDs = smartLocation.drop(columns=['OBJECTID', 'GEOID10', 'TRFIPS', 'CFIPS', 'SFIPS', 'CSA','CSA_Name','CBSA','CBSA_Name','CBSA_EMP','CBSA_POP','CBSA_WRK','COUNTHU10','TOTPOP10','HH','AUTOOWN0','AUTOOWN1','AUTOOWN2P','WORKERS','R_LOWWAGEWK','R_MEDWAGEWK','R_HIWAGEWK','EMPTOT','E_FEDT10','E_FEDRET10','E_FEDOFF10','E_FEDIND10','E_FEDSVC10','E_FEDENT10','E_LOWWAGEW','E_MEDWAGEW','E_HIWAGEWK','AC_TOT','AC_WATER','AC_LAND','AC_UNPR','Shape_Length', 'Shape_Area'])

In [9]:
#clean smartLocations manually
smartLocation.head()
#smartLocationClean = smartLocation.drop(columns=['OBJECTID','CSA_Name','CBSA_Name','AUTOOWN0','AUTOOWN1','AUTOOWN2','WORKERS','R_LOWWAGEWK','R_MEDWAGEWK','R_HIWAGEWK','EMPTOT','Shape_Length','Shape_Area'])
#smartLocation.head()

,OBJECTID,GEOID10,TRFIPS,CFIPS,SFIPS,CSA,CSA_Name,CBSA,CBSA_Name,CBSA_EMP,...,D5cr,D5cri,D5ce,D5cei,D5dr,D5dri,D5de,D5dei,Shape_Length,Shape_Area
0,1,10059505002,950500,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.04,0.66,4.24,0.68,-99999.0,-99999.0,-99999.0,-99999.0,0.846952,0.018273
1,2,10059505001,950500,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.04,0.67,4.70,0.75,-99999.0,-99999.0,-99999.0,-99999.0,0.521148,0.005095
2,3,10059502001,950200,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.02,0.30,2.60,0.42,-99999.0,-99999.0,-99999.0,-99999.0,0.660835,0.018740
3,4,10059502002,950200,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.01,0.27,2.58,0.41,-99999.0,-99999.0,-99999.0,-99999.0,0.695057,0.020340
4,5,10059504002,950400,5,1,NaN,NaN,21640.0,"Eufaula, AL-GA",9921.0,...,0.02,0.32,3.35,0.54,-99999.0,-99999.0,-99999.0,-99999.0,0.499117,0.008829


In [97]:
mergedData = pd.merge(modeShareSOVPerc, smartLocation, left_on='Geo_FIPS', right_on='GEOID10')
print(len(mergedData))
mergedData = mergedData.dropna(subset=['percSOV'])
print(len(mergedData))

39611
39460


In [101]:
#print(list(mergedData))
#for i in mergedData:
 # isNull = list(pd.isna(mergedData[i]))
  #if(isNull.count(True)!=0):
   #     print(i, "has", isNull.count(True), "null values")
  #print(isNull.count(True))
a = mergedData['CBSA_Name']
print(mergedData.columns.get_loc('CBSA_Name'))
print(len(mergedData.columns))
mergedData['CBSA_Str'] = 'a'
print(len(mergedData.columns))
#print(mergedData.iloc[:,119])
#print(mergedData.iloc[0,119])
a = list(mergedData)
#for i in range(len(mergedData)):
 # mergedData.iloc[i, 119] = str(mergedData.iloc[i,10])
print(mergedData.iloc[:, 119])
#print(b)
#b.describe()

10
120
120
0        a
1        a
2        a
3        a
4        a
5        a
6        a
7        a
8        a
9        a
10       a
11       a
12       a
13       a
14       a
15       a
16       a
17       a
18       a
19       a
20       a
21       a
22       a
23       a
24       a
25       a
26       a
27       a
28       a
29       a
        ..
39581    a
39582    a
39583    a
39584    a
39585    a
39586    a
39587    a
39588    a
39589    a
39590    a
39591    a
39592    a
39593    a
39594    a
39595    a
39596    a
39597    a
39598    a
39599    a
39600    a
39601    a
39602    a
39603    a
39604    a
39605    a
39606    a
39607    a
39608    a
39609    a
39610    a
Name: CBSA_Str, Length: 39460, dtype: object


In [11]:
#converts mergedData to csv so that it can be cleaned
mergedData.to_csv("mergedData.csv")

In [12]:
mergedDataClean = pd.read_csv("mergedData.csv")

In [13]:
#mergedDataClean = mergedDataClean.drop(columns=["CSA_Name"])
isNull = list(pd.isna(mergedDataClean["CSA"]))
#print(isNull)
print(isNull.count(True))


13992


In [14]:
#Here is where all the actual Darwin things start 
#Set your user id and password accordingly
from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

USER="nsusanward@gmail.com"
PW="G8adjr4PCu"

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET=''
DATASET_NAME='mergedData.csv'

# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [15]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)

In [39]:
# Preview dataset
#df = pd.read_csv(os.path.join(PATH_TO_DATASET, DATASET_NAME))
#df.head()
#print(list(df.isnull().sum()))
#print(max(df.isnull().sum())-len(df))
#print(list(df))
#print(df.iloc[:,0])
display(df[['Geo_FIPS','CSA_Name']])

,Geo_FIPS,CSA_Name
0,10010201001,"Montgomery-Alexander City, AL"
1,10010201002,"Montgomery-Alexander City, AL"
2,10010202001,"Montgomery-Alexander City, AL"
3,10010202002,"Montgomery-Alexander City, AL"
4,10010203001,"Montgomery-Alexander City, AL"
5,10010203002,"Montgomery-Alexander City, AL"
6,10010204001,"Montgomery-Alexander City, AL"
7,10010204002,"Montgomery-Alexander City, AL"
8,10010204003,"Montgomery-Alexander City, AL"
9,10010204004,"Montgomery-Alexander City, AL"


In [17]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, DATASET_NAME))
if not status:
    print(dataset)

400: BAD REQUEST - {"message": "Dataset already exists"}



In [18]:
# clean dataset
target = "percSOV"
status, job_id = ds.clean_data(DATASET_NAME, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
    print("hello")
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-03T20:12:33.066663', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['d2937659183e4775bb1b6bb933509e9a'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-03T20:12:33.066663', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['d2937659183e4775bb1b6bb933509e9a'], 'model_name': None, 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-03T20:12:33.066663', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['d2937659183e4775bb1b6bb933509e9a'], 'model_name': None, 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-03T20:12:33.066663', 'endtime': '2019-04-03T20:13:06.0

In [1]:
mergedDataClean.head()

NameError: name 'mergedDataClean' is not defined